In [2]:
!yes | pip uninstall torchvison
!pip install -qU torchvision

yes: standard output: Broken pipe
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
# prefix = "sagemaker/pytorch-mnist"
prefix = "sagemaker/ucsc805"

role = sagemaker.get_execution_role()

In [4]:
###   Bobby test import of deepset/roberta-base-squad2

import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'deepset/roberta-base-squad2',
	'HF_TASK':'question-answering'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.26.0',
	pytorch_version='1.13.1',
	py_version='py39',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	# instance_type='ml.m5.xlarge' # ec2 instance type
    instance_type='ml.m5.xlarge' # ec2 instance type
)

predictor.predict({
	"inputs": {
		"question": "What is my name?",
		"context": "My name is Clara and I live in Berkeley."
	},
})

-----!

{'score': 0.9331287145614624, 'start': 11, 'end': 16, 'answer': 'Clara'}

In [5]:
###
predictor.predict({
	"inputs": {
		"question": "Where do I live?",
		"context": "My name is Clara and I live in Berkeley."
	}
})

{'score': 0.9396458864212036, 'start': 31, 'end': 39, 'answer': 'Berkeley'}

In [6]:
###
predictor.predict({
	"inputs": {
		"question": "what country do I live in?",
		"context": "My name is Clara and I live in Berkeley, CA."
	}
})

{'score': 0.06728031486272812, 'start': 41, 'end': 43, 'answer': 'CA'}

In [7]:
###
# !pip install transformers

In [8]:
# !pip install ucimlrepo

In [9]:
# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# adult = fetch_ucirepo(id=2) 
  
# # data (as pandas dataframes) 
# X = adult.data.features 
# y = adult.data.targets 
  
# # metadata 
# print(adult.metadata) 
  
# # variable information 
# print(adult.variables) 


In [10]:
!pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d3/95/ef83542e7a8e2bfc4432ee2cd8a6b52eb30fb1e605871e8871e94ce65fb1/datasets-2.13.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tqdm>=4.62.1 from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 844.1 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/54/a0/dae1c5dc27601a61897b48a367232c743c760c765d9ab38be1a903cf0d87/xxhash-3.4.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0.0,>=0.11.0 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a4

In [11]:
###
# import torch
import datasets
from datasets import load_dataset
# from torchvision.datasets import MNIST, load_dataset
# from torchvision import  load_dataset
# from torchvision import transforms

dataset = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [13]:
print("number of records in train Dataset: ", len(dataset['train']))
print("number of records in validation Dataset: ", len(dataset['validation']))

print("accessing value at index 1 of the simple_dataset object: ", dataset['train'][1])

number of records in train Dataset:  130319
number of records in validation Dataset:  11873
accessing value at index 1 of the simple_dataset object:  {'id': '56be85543aeaaa14008c9065', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'What areas did Beyonce compete in when she was growing up?', 'answers'

# @article{2016arXiv160605250R,
#        author = {{Rajpurkar}, Pranav and {Zhang}, Jian and {Lopyrev},
#                  Konstantin and {Liang}, Percy},
#         title = "{SQuAD: 100,000+ Questions for Machine Comprehension of Text}",
#       journal = {arXiv e-prints},
#          year = 2016,
#           eid = {arXiv:1606.05250},
#         pages = {arXiv:1606.05250},
# archivePrefix = {arXiv},
#        eprint = {1606.05250},
# }

In [ ]:
# from torchvision.datasets import MNIST
# from torchvision import transforms

# MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

# MNIST(
#     "lab03-pytorch-data",
#     download=True,
#     transform=transforms.Compose(
#         [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
#     ),
# )

## Data
### Getting the data



### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [ ]:
inputs = sagemaker_session.upload_data(path="lab03-pytorch-data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [ ]:
# !pygmentize mnist-pytorch.py

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist-pytorch.py` script above.


In [ ]:
# from sagemaker.pytorch import PyTorch

# estimator = PyTorch(
#     entry_point="mnist-pytorch.py",
#     role=role,
#     py_version="py38",
#     framework_version="1.11.0",
#     instance_count=2,
#     instance_type="ml.c5.2xlarge",
#     hyperparameters={"epochs": 1, "backend": "gloo"},
)

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
# estimator.fit({"training": inputs})

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist-pytorch.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist-pytorch.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [ ]:
# predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

In [ ]:
# !ls lab03-pytorch-data/MNIST/raw

In [ ]:
# import gzip
# import numpy as np
# import random
# import os

# data_dir = "lab03-pytorch-data/MNIST/raw"
# with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
#     images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

# mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
# mask = np.array(mask, dtype=int)
# data = images[mask]

In [ ]:
# response = predictor.predict(np.expand_dims(data, axis=1))
# print("Raw prediction result:")
# print(response)
# print()

# labeled_predictions = list(zip(range(10), response[0]))
# print("Labeled predictions: ")
# print(labeled_predictions)
# print()

# labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
# print("Most likely answer: {}".format(labeled_predictions[0]))

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
# sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)